In [2]:
import pandas as pd
import numpy as np
agg_df = pd.read_csv('Migros_case/triplets.csv')

In [3]:
R_df = agg_df.pivot(index = 'user_id', columns ='product_id', values = 'amount').fillna(0)
R_df.head()

product_id,71201400505,74300100905,74300300505,74300300605,74300302105,74300302205,74300400805,74300500205,74300603005,74400100105,...,999999901197,999999901260,999999901273,999999901274,999999901284,999999901287,999999901288,999999901290,999999901291,999999901299
user_id,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
R = R_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [5]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 100)
sigma = np.diag(sigma)


In [6]:
original_values = R_df.copy()

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
all_user_predicted_ratings[all_user_predicted_ratings < 0] = 0
predicted_df = pd.DataFrame(all_user_predicted_ratings, index=R_df.index, columns=R_df.columns)

mask = (original_values == 0)
original_values[mask] = predicted_df[mask]
final_predictions = original_values



In [13]:
final_predictions.to_pickle('Migros_case/predicted_triplets.pkl')

In [10]:
rows, cols = np.where(final_predictions > 1)
for row, col in zip(rows, cols):
    print(f"user_id: {final_predictions.index[row]}, product_id: {final_predictions.columns[col]}, value: {final_predictions.iloc[row, col]}")
    print(f"{agg_df[(agg_df['user_id'] == final_predictions.index[row]) & (agg_df['product_id'] == final_predictions.columns[col])].head(1).amount.values[0]}")


user_id: 100001, product_id: 104202000000, value: 2.0
2.0
user_id: 100001, product_id: 106225100000, value: 2.0
2.0
user_id: 100001, product_id: 115027200000, value: 3.0
3.0
user_id: 100001, product_id: 122054000000, value: 2.0
2.0
user_id: 100001, product_id: 200609700000, value: 2.0
2.0
user_id: 100001, product_id: 204000600400, value: 11.0
11.0
user_id: 100001, product_id: 204001000000, value: 4.0
4.0
user_id: 100001, product_id: 204001100000, value: 8.0
8.0
user_id: 100001, product_id: 204130100000, value: 3.0
3.0
user_id: 100001, product_id: 205062200400, value: 3.0
3.0
user_id: 100001, product_id: 231502083302, value: 2.0
2.0
user_id: 100001, product_id: 241001092000, value: 1.871
1.871
user_id: 100001, product_id: 260215701000, value: 2.758
2.758
user_id: 100001, product_id: 263382401000, value: 1.024
1.024
user_id: 100001, product_id: 264200301000, value: 1.783
1.783
user_id: 100001, product_id: 271708513200, value: 2.0
2.0
user_id: 100001, product_id: 273708902100, value: 2.46

IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
# find user 100001 prediction
final_predictions.loc[100001].sort_values(ascending=False)

product_id
204000600400    11.0
204001100000     8.0
204001000000     4.0
115027200000     3.0
205062200400     3.0
                ... 
516341000000     0.0
516344000000     0.0
131243300000     0.0
131242900700     0.0
999999901299     0.0
Name: 100001, Length: 31364, dtype: float64

In [21]:
df

amount
user_id product_id        
1       A                5
        B                3
2       A                4
        C                2
3       B                3
        C                6

In [2]:
import pandas as pd
final_predictions = pd.read_pickle('Migros_case/predicted_triplets.pkl')
#select small portion of final_predictions
#final_predictions = final_predictions.iloc[:1000, :1000]

In [3]:
from sklearn.metrics.pairwise import cosine_similarity


similarity_matrix = cosine_similarity(final_predictions)
similarity_df = pd.DataFrame(similarity_matrix, index=final_predictions.index, columns=final_predictions.index)
similarity_df.head()

user_id,100001,100002,100003,100004,100005,100006,100007,100008,100009,100010,...,109991,109992,109993,109994,109995,109996,109997,109998,109999,110000
user_id,,,,,,,,,,,,,,,,,,,,,
100001,1.000000,0.051394,0.039444,0.059669,0.050319,0.051323,0.076467,0.083717,0.128889,0.011336,...,0.088118,0.023783,0.024676,0.033358,0.049146,0.049942,0.039145,0.053321,0.050713,0.002499
100002,0.051394,1.000000,0.087256,0.105334,0.065153,0.039834,0.034681,0.068456,0.106991,0.029516,...,0.083774,0.019279,0.048306,0.029259,0.072830,0.061877,0.049378,0.062337,0.072194,0.014679
100003,0.039444,0.087256,1.000000,0.101910,0.060327,0.034615,0.052869,0.065026,0.070234,0.018237,...,0.095528,0.029317,0.079799,0.046361,0.062241,0.061341,0.052031,0.050003,0.054293,0.009597
100004,0.059669,0.105334,0.101910,1.000000,0.076302,0.043959,0.093374,0.180700,0.107386,0.042189,...,0.193136,0.071791,0.142003,0.089902,0.087287,0.085496,0.069857,0.078450,0.066130,0.019253
100005,0.050319,0.065153,0.060327,0.076302,1.000000,0.024901,0.054092,0.082506,0.062484,0.024553,...,0.062948,0.018426,0.043867,0.021306,0.062199,0.063958,0.062404,0.059087,0.037903,0.008664


In [69]:
#in the similarity_df the diagonal should be -1
np.fill_diagonal(similarity_df.values, -1)
similarity_df.to_pickle('Migros_case/similarity_df.pkl')

In [34]:
score_df

,Product_number,Product_description,M-Check Packaging
0,274460003300,Kartoffeln festkochend 2.5kg,5.0
1,274450003300,Kartoffeln mehligkochend 2.5kg,5.0
2,265384102300,Bio Trauben gemischt kernlos,4.0
3,265383002300,Bio Trauben weiss kernlos,4.0
4,235300383000,Bio Pastrami MAXI,2.0
...,...,...,...
1183,130931500000,Bio Aha Tofu Zitronen/Thymian 250g,4.0
1184,130931400000,Bio Aha Tofu Basilikum 250g,4.0
1185,101404600000,Bio CH-Alpenkräuter Nussmischung,3.0
1186,101416900000,M-Sélection Mandeln Piment d'Espelette Ltd.,3.0


In [50]:
score_df = pd.read_csv('Migros_case/M-Check_packaging.csv', sep=";",encoding='latin-1')
# drop rows that only contains NaN
score_df.dropna(how='all', inplace=True)
#rename column calle dproduct number to product id
score_df.rename(columns={'Product_number': 'product_id'}, inplace=True)


# set product id as int
score_df['product_id'] = score_df['product_id'].astype('string')

# red triplets.csv  (user_id,product_id,amount) and get most popular products (not based on the amount but the number product_id occurs) and normalise the count from 0 to 1
triplets_df = pd.read_csv('Migros_case/triplets.csv')
grouped_triplets_df = triplets_df.groupby('product_id').count().reset_index()
grouped_triplets_df['amount'] = grouped_triplets_df['user_id'].apply(lambda x: x / grouped_triplets_df['user_id'].sum())
grouped_triplets_df.drop('user_id', axis=1, inplace=True)

# set proiduct id as string
grouped_triplets_df['product_id'] = grouped_triplets_df['product_id'].astype('string')

# emrge it with score_df
score_df = score_df.merge(grouped_triplets_df, on='product_id', how='left').fillna(0)
#rename amount to popularity
score_df.rename(columns={'amount': 'popularity'}, inplace=True)
score_df.to_csv("Migros_case/Score_popularity.csv")

In [51]:
#read score_popularity
score_df = pd.read_csv('Migros_case/Score_popularity.csv',index_col=0)
score_df

,product_id,Product_description,M-Check Packaging,popularity
0,274460003300,Kartoffeln festkochend 2.5kg,5.0,0.001788
1,274450003300,Kartoffeln mehligkochend 2.5kg,5.0,0.000762
2,265384102300,Bio Trauben gemischt kernlos,4.0,0.000000
3,265383002300,Bio Trauben weiss kernlos,4.0,0.000118
4,235300383000,Bio Pastrami MAXI,2.0,0.000014
...,...,...,...,...
1183,130931500000,Bio Aha Tofu Zitronen/Thymian 250g,4.0,0.000000
1184,130931400000,Bio Aha Tofu Basilikum 250g,4.0,0.000000
1185,101404600000,Bio CH-Alpenkräuter Nussmischung,3.0,0.000000
1186,101416900000,M-Sélection Mandeln Piment d'Espelette Ltd.,3.0,0.000000


In [41]:
#sort triplets_df by amount
triplets_df.sort_values('amount', ascending=True).head()

,user_id,product_id,amount
150497,101885,220234000040,0.002
467458,105916,264683101000,0.008
756569,109518,271711501000,0.010
49958,100630,271711501000,0.010
263984,103344,271711501000,0.010


### Select top sustainable products from most isimilalr user

In [ ]:
# check if user id 100131500000 and user id 

In [73]:
import numpy as np


def get_product_from_similar_user(sorted_user_similarities, triplets_df,userID,num_users=1):
    # get num_users most similar users
    most_similar_users = sorted_user_similarities.iloc[0:num_users].index.tolist()

    print('Similar user: ',most_similar_users)

    # get all product of userId and all products of user_row_number (they are in triplets_df)
    most_similar_user_products = triplets_df[triplets_df['user_id'].isin(most_similar_users)]['product_id'].tolist()

    return most_similar_user_products


def recommend_similar_users_sustainable_products(predictions_df, userID, score_df, triplets_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    sorted_user_similarities = predictions_df.loc[userID].sort_values(ascending=False)

    # get products of most similar user by looking at 
    most_similar_user_products = get_product_from_similar_user(sorted_user_similarities, triplets_df,userID,num_users=1)

    # get all product of userId and all products of user_row_number (they are in triplets_df)
    userID_products= triplets_df[triplets_df['user_id'] == userID]['product_id'].tolist()
    # get products that are in most_similar_user_products but not in userID_products
    products_to_recommend = np.setdiff1d(most_similar_user_products, userID_products)

    
    #sort by sustainabiltiy score (the score is in score_df)
    products_to_recommend = products_to_recommend[np.argsort(score_df[score_df['product_id'].isin(products_to_recommend)]['M-Check Packaging'])]

    if products_to_recommend.size == 0:
        # if there is no product, look into the top 3 similar users
        most_similar_user_products = get_product_from_similar_user(sorted_user_similarities, triplets_df,userID,num_users=3)
        products_to_recommend = np.setdiff1d(most_similar_user_products, userID_products)
        products_to_recommend = products_to_recommend[np.argsort(score_df[score_df['product_id'].isin(products_to_recommend)]['M-Check Packaging'])]

        if products_to_recommend.size == 0:
        
            # just give popular sustainable products (popular based on popularity, sustainable based on 'mcheckpackaging). Suistanable must have priority
            products_to_recommend = score_df.sort_values(['M-Check Packaging', 'popularity'], ascending=[False, False])['product_id'].tolist()
        
    # return top 5
    return products_to_recommend[:num_recommendations]

score_df = pd.read_csv('Migros_case/Score_popularity.csv',index_col=0)
score_df.dropna(how='all', inplace=True)
#put into string
score_df['product_id'] = score_df['product_id'].astype('string')

triplets_df = pd.read_csv('Migros_case/triplets.csv')
#put all into string
triplets_df['product_id'] = triplets_df['product_id'].astype('string')
triplets_df['user_id'] = triplets_df['user_id'].astype('string')

similarity_df = pd.read_pickle('Migros_case/similarity_df.pkl')
#put into string
similarity_df.index = similarity_df.index.astype('string')
similarity_df.columns = similarity_df.columns.astype('string')

recommended_products = recommend_similar_users_sustainable_products(similarity_df, '100001', score_df, triplets_df, 5)
recommended_products



Similar user:  ['108376']


array(['100500100000', '100500200000', '101020100000', '101055900000',
       '101911200000'], dtype='<U12')